In [1]:
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [2]:
subs = ['wsb', 'science', 'ama', 'askreddit', 'worldnews', 'funny', 'dankmemes', 'memes', 'nosleep', 'psychology', 'politics', 'cscareerquestions', 'writingprompts']
data = {}
for i, c in enumerate(subs):
    with open("/home/nick/Documents/Reddit_NLP/pickles/" + c + ".pickle", "rb") as file:
        data[c] = pickle.load(file)

In [3]:
data['wsb']

['In the future when Apple is at 50000 a share your gran kids will ask why you didn’t buy Apple at 400. And you’ll answer with “I bought corn instead”.',
 'You ever stare at your account and think: wow, how did I put $20k into options and think that was safe?',
 'Buy [TSLA](https://imgur.com/a/ZWp53bV) at the top gang. 🤡',
 'Almost responded to an email with “thanks, retard” because 50% of my social interaction is now through this sub',
 '##Top trending tickers, sentiments, and options on /r/wallstreetbets for the day of August 12, 2020.\n\n────────────────\n\n###Trending Tickers:\n\n**Ticker**|**Company**|**Mentions**|**Bulls / Bears**|**Price**|**Change**|**P/E**\n:-:|:--|:-:|:-:|:-:|:-:|:-:\nTSLA|Tesla, Inc.|783|80.72% / 19.28%|$1552.15|12.93%|707.35\nSPY|Exchange Traded Fund|433|49.29% / 50.71%|$337.65|1.46%|-\nSLV|Exchange Traded Fund|252|87.97% / 12.03%|$23.70|1.61%|-\nAAPL|Apple Inc.|250|84.11% / 15.89%|$451.98|3.31%|33.25\nCORN|Exchange Traded Fund|181|92.11% / 7.89%|$11.72|0.4

In [4]:
def clean(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', ' ', text)
    return text

In [5]:
def pre_clean(documents):
    cleaned_list = []
    for doc in documents:
        cleaned_list.append(clean(doc))
    return cleaned_list

In [6]:
# def lemmatize_take_string(text):
#     nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
#     nlp.max_length = 14082871
#     teckst = nlp(text)
#     return ' '.join([word.lemma_ for word in teckst if not word.is_stop])

In [7]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp(data['wsb'][0])

In the future when Apple is at 50000 a share your gran kids will ask why you didn’t buy Apple at 400. And you’ll answer with “I bought corn instead”.

In [8]:
def lemmatize(text):
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    nlp.max_length = 14082871 
    result = nlp(text) 
    lemmed = [word.lemma_ for word in result]
    return lemmed

In [9]:
def pre_lemma(documents):
    lemmed_list = []
    for doc in documents:
        lemmed_list.append(lemmatize(doc))
    return lemmed_list

In [10]:
for key, value in data.items():
    data[key] = pre_clean(value)

data['wsb']

['in the future when apple is at  a share your gran kids will ask why you didnt buy apple at  and youll answer with i bought corn instead',
 'you ever stare at your account and think wow how did i put  into options and think that was safe',
 'buy  at the top gang 🤡',
 'almost responded to an email with thanks retard because  of my social interaction is now through this sub',
 'top trending tickers sentiments and options on rwallstreetbets for the day of august    ────────────────  trending tickers  tickercompanymentionsbulls  bearspricechangepe  tslatesla    spyexchange traded    slvexchange traded    aaplapple    cornexchange traded    rktrocket companies    msftmicrosoft    amdadvanced micro    bathe boeing    fbfacebook    prplpurple innovation    qqqexchange traded    netcloudflare    cscocisco systems    nvdanvidia      ────────────────  trending options  tickerstriketypeexpiration                   ────────────────  trending sentiment  sentimentcomments       ────────────────  ot

In [11]:
data2 = data.copy()
data2

{'wsb': ['in the future when apple is at  a share your gran kids will ask why you didnt buy apple at  and youll answer with i bought corn instead',
  'you ever stare at your account and think wow how did i put  into options and think that was safe',
  'buy  at the top gang 🤡',
  'almost responded to an email with thanks retard because  of my social interaction is now through this sub',
  'top trending tickers sentiments and options on rwallstreetbets for the day of august    ────────────────  trending tickers  tickercompanymentionsbulls  bearspricechangepe  tslatesla    spyexchange traded    slvexchange traded    aaplapple    cornexchange traded    rktrocket companies    msftmicrosoft    amdadvanced micro    bathe boeing    fbfacebook    prplpurple innovation    qqqexchange traded    netcloudflare    cscocisco systems    nvdanvidia      ────────────────  trending options  tickerstriketypeexpiration                   ────────────────  trending sentiment  sentimentcomments       ────────

In [15]:
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data, orient='index').fillna(value='none')
# data_df.columns = ['text']
# data_df = data_df.sort_index()
data_df

,0,1,2,3,4,5,6,7,8,9,...,101131,101132,101133,101134,101135,101136,101137,101138,101139,101140
wsb,in the future when apple is at a share your gran kids will ask why you didnt buy apple at and youll answer with i bought corn instead,you ever stare at your account and think wow how did i put into options and think that was safe,buy at the top gang 🤡,almost responded to an email with thanks retard because of my social interaction is now through this sub,top trending tickers sentiments and options on rwallstreetbets for the day of august ──────────────── trending tickers tickercompanymentionsb...,some year old corn futures trader whos been asleep at his desk for years is just like wtf are all these options 🤨,blew my account up today feels bad lost over hopping between trades feels bad,overnight gap up to ath secured,usually my balls tingle the day before a significant pullback so i think bulls are safe for now,wsb after dark is going to be wild tonight i heard were getting strippers,...,none,none,none,none,none,none,none,none,none,none
science,the children were not motivated by achieving the maximum reward to the extent that adults were blanco said instead children seemed primarily moti...,also proved children like to gamble according to the study,kind of like how gamers go through rpg games explore systematically,i teach elementary pe to year olds one of the big early breakthroughs i had in my teaching came around this idea i have very specific standards ...,wouldnt this make lootmystery boxes even worse when aimed to children they wouldnt even care about the odds they just have to know what will they get,i find that very encouraging,you could have a boat or whats in this box,like when playing diablo scanned every dungeon everytime,i have a and year old boy by not shoving screens in front of them they spend of their day in the backyardwhich is big exploring and playing the...,basically how ppl play rpgs they explore everything to make sure they havent missed anything,...,none,none,none,none,none,none,none,none,none,none
ama,tldr do whatever the fuck you want,so youre telling me people on the internet lie,,this is so funny this subreddit got fucked by one person,a note about celebrity request bans when riama thats the bigger sub had its peak wave of popularity about ten years ago that sub and therefore thi...,do you like cantaloupe,what does ama stand for,i liked that guy he seemed like a really nice person,hmmm ok thnx for clarifying all that but one quick q did cancer kid get like verified or anything before he posted,so basically rama is a shitty knockoff of rima meant for telling your fake stories or just shit posting nice,...,none,none,none,none,none,none,none,none,none,none
askreddit,victorias secret was originally supposed to be a place for men to buy lingerie for their wives and not feel embarrassed but now its almost exclusi...,the ford mustang it was supposed to be a car with a big trunk to you could fit all the groceries in it there was even an ad or article stating t...,heels if i remember correctly,is this how family feud gets new questions and answers,skirts used to be worn in battle by men for more room to move around,ugg boots were originally worn by male surfers with their girlfriends stealing and wearing them like they would a leather jacket,filtered cigarettes in general but marlboro made the biggest leap in terms of gendered marketing once that whole conspiracy theory came out that l...,those leg stirrups they use at the gynocologist were invented for removing kidney stones in men they used to cut from below the testicles and cut ...,my hoodie apparently,pink and blue were originally considered colors for boys and girls respectively,...,shut up and take all my caps,is that butcher pete i see,happy cake day and yes maybe at bars close to mit,heppy cake day,not his ghost tho,maybe,is it my cake day i have always wondered how people know that,theres a cake emoji next to your username on your comments when its you

In [ ]:
for key, value in data.items():
    data[key] = pre_lemma(value)
    
data['wsb']

In [ ]:
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data, orient='index').fillna(value='none')
# data_df.columns = ['text']
# data_df = data_df.sort_index()
data_df

In [ ]:
pd.set_option('max_colwidth',150)

data_df_nolemma = pd.DataFrame.from_dict(data2, orient='index').fillna(value='none')
# data_df.columns = ['text']
# data_df = data_df.sort_index()
data_df_nolemma

In [ ]:
import multiprocessing
from gensim.models import Word2Vec
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

In [ ]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [ ]:
from time import time
t = time()

w2v_model.build_vocab(data.values(), progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
t = time()

w2v_model.train(data.values(), total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
w2v_model.init_sims(replace=True)

In [ ]:
w2v_model.wv.similarity('take', 'give')

In [ ]:
w2v_model.wv.similarity('science', 'career')

In [ ]:
w2v_model.wv.doesnt_match(['bad', 'three', 'two'])

In [ ]:
w2v_model.wv.most_similar(positive=["europe", "america"], negative=["russia"], topn=3)

In [ ]:
w2v_model.wv.most_similar(positive=["europe", "america"], negative=["china"], topn=3)

In [ ]:
w2v_model.wv.most_similar(positive=["europe", "america"], negative=["britain"], topn=3)

In [ ]:
w2v_model.wv.most_similar(positive=["app", "twitter"], negative=["app"], topn=3)